# Tranformaciones ETL

Importamos las librerias

In [6]:
import pandas as pd
import numpy as np
import json
import ast

Cargamos los csv

In [10]:
df=pd.read_csv(r"C:\Users\WINDOW 10\Desktop\Labs 2\Datasets\movies_dataset.csv")
df2 = pd.read_csv(r"C:\Users\WINDOW 10\Desktop\Labs 2\Datasets\credits.csv")

C:\Users\WINDOW 10\AppData\Local\Temp\ipykernel_25372\3302552323.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\WINDOW 10\Desktop\Labs 2\Datasets\movies_dataset.csv")


In [8]:
#verifico que cargo correctamente
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [11]:
def merge_movies_and_credits(df, df2):
    '''
    La función devuelve el DataFrame resultado de unir movies y credits. Para ello elimina los id's que no están en
    el formato adecuado (número entero), elimina los id's duplicados y finalmente los convierte a enteros de 64 bits.
    '''
    
    # Convierte los id's de cada DataFrame a entero y si no se puede lo etiqueta con NaN
    df['id'] = pd.to_numeric(df['id'], errors='coerce')
    df2['id'] = pd.to_numeric(df2['id'], errors='coerce')
    
    # Dropea los registros que son NaN
    df = df.dropna(subset=['id'])
    df2 = df2.dropna(subset=['id'])
    
    # Dropea los registros duplicados
    df.drop_duplicates(subset='id', inplace=True)
    df2.drop_duplicates(subset='id', inplace=True) 

    # Juntar los DataFrames utilizando el campo "id"
    merged_data = pd.merge(df, df2, on='id')
    
    # Eliminar los registros que no tienen título 
    merged_data = merged_data.dropna(subset=['title'])
    
    # Restablecer el índice
    merged_data = merged_data.reset_index(drop=True)
    
    return merged_data


df = merge_movies_and_credits(df, df2)

C:\Users\WINDOW 10\AppData\Local\Temp\ipykernel_25372\927150050.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset='id', inplace=True)


In [12]:
#Elimino las columnas que no se utilizara
df.drop(columns=['video','imdb_id','adult','original_title','poster_path','homepage'], inplace=True)

In [13]:
#Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0
df['revenue']=df['revenue'].fillna(0)
df['budget']=df['budget'].fillna(0)

In [14]:
#Convertimos revenue y budget a float 
df['revenue'] = df['revenue'].replace('[^\d.]', '', regex=True).astype(float)
df['budget'] = df['budget'].replace('[^\d.]', '', regex=True).astype(float)

In [15]:
#crear la columna return para calcular la ganacia
df['return'] = df.apply(lambda row: 0 if row['budget'] == 0 else row['revenue']/row['budget'], axis=1)

In [16]:
df=df.dropna(subset=['release_date'])

De haber fechas, deberán tener el formato AAAA-mm-dd

In [17]:
df['release_date']=pd.to_datetime(df['release_date'],format='%Y-%m-%d',errors='coerce')

In [18]:
df['release_date'].isna().sum()

0

In [19]:
df[df['release_date'].isna()]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,return


In [20]:
#Dropeamos los valores nulos
df=df.dropna(subset=['release_date'])

Creamos la columna release_year donde se extraera el año de la fecha de estreno.

In [21]:
df['release_year']=df['release_date'].dt.year

In [22]:
df['release_year'] = pd.DatetimeIndex(df['release_date']).year
df['release_year'].head(10)

0    1995
1    1995
2    1995
3    1995
4    1995
5    1995
6    1995
7    1995
8    1995
9    1995
Name: release_year, dtype: int64

In [23]:
#Verificamos nuestro DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45345 entries, 0 to 45428
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4485 non-null   object        
 1   budget                 45345 non-null  float64       
 2   genres                 45345 non-null  object        
 3   id                     45345 non-null  float64       
 4   original_language      45334 non-null  object        
 5   overview               44404 non-null  object        
 6   popularity             45345 non-null  object        
 7   production_companies   45345 non-null  object        
 8   production_countries   45345 non-null  object        
 9   release_date           45345 non-null  datetime64[ns]
 10  revenue                45345 non-null  float64       
 11  runtime                45099 non-null  float64       
 12  spoken_languages       45345 non-null  object        
 13  s

Algunos campos, como belongs_to_collection, production_companies Y genres están anidados 

In [24]:
#Importamos JSON
import json

In [25]:
def new_name(x):
    try:
        return json.loads(x.replace("'", "\""))
    except (json.JSONDecodeError, AttributeError):
        return {}

df["belongs_to_collection"] = df["belongs_to_collection"].apply(new_name)
df["collection_id"] = df["belongs_to_collection"].apply(lambda x: x["id"] if isinstance(x, dict) and "id" in x else None)
df["collection_name"] = df["belongs_to_collection"].apply(lambda x: x["name"] if isinstance(x, dict) and "name" in x else None)
df["collection_poster"] = df["belongs_to_collection"].apply(lambda x: x["poster_path"] if isinstance(x, dict) and "poster_path" in x else None)
df["collection_backdrop"] = df["belongs_to_collection"].apply(lambda x: x["backdrop_path"] if isinstance(x, dict) and "backdrop_path" in x else None)
df.drop(['belongs_to_collection'],axis=1,inplace=True)

Creamos la misma Funcion para belongs_to_collection, production_companies

In [26]:
def new_name(json_string):
    try:
        return json.loads(json_string.replace("'", "\""))
    except:
        return {}

In [27]:
# Aplicar la función a la columna "genres"
df["genres"] = df["genres"].apply(new_name)
df["genre_ids"] = df["genres"].apply(lambda x: [d.get("id") for d in x])
df["genre_names"] = df["genres"].apply(lambda x: [d.get("name") for d in x])
df["genre_names"].head(10)

0           [Animation, Comedy, Family]
1          [Adventure, Fantasy, Family]
2                     [Romance, Comedy]
3              [Comedy, Drama, Romance]
4                              [Comedy]
5      [Action, Crime, Drama, Thriller]
6                     [Comedy, Romance]
7    [Action, Adventure, Drama, Family]
8         [Action, Adventure, Thriller]
9         [Adventure, Action, Thriller]
Name: genre_names, dtype: object

In [28]:
df["production_companies"] = df["production_companies"].apply(new_name)
df["production_companies_ids"] = df["production_companies"].apply(lambda x: [d.get("id") for d in x])
df["production_companies_names"] = df["production_companies"].apply(lambda x: [d.get("name") for d in x])

In [29]:
# Aplicamos la funcion a 'production_countries'
df["country"] = df["production_countries"].apply(lambda x: new_name(x))
df["country"] = df["country"].apply(lambda x: x[0]["name"] if isinstance(x, list) and len(x) > 0 else None)

In [30]:
#Aplicamos la funcion a Spoken Lenguages
df["spoken_languages"] = df["spoken_languages"].apply(lambda x: new_name(x))
df["spoken_languages"] = df["spoken_languages"].apply(lambda x: x[0]["name"] if isinstance(x, list) and len(x) > 0 else None)


In [31]:
#Borramos los NUll de las siguientes columnas

df.drop(['genres','production_companies','production_countries'],axis=1,inplace=True)

In [32]:
#Los valores nulos del campo release date deben eliminarse.
df.dropna(subset=['release_date'],inplace=True)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45345 entries, 0 to 45428
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   budget                      45345 non-null  float64       
 1   id                          45345 non-null  float64       
 2   original_language           45334 non-null  object        
 3   overview                    44404 non-null  object        
 4   popularity                  45345 non-null  object        
 5   release_date                45345 non-null  datetime64[ns]
 6   revenue                     45345 non-null  float64       
 7   runtime                     45099 non-null  float64       
 8   spoken_languages            41554 non-null  object        
 9   status                      45265 non-null  object        
 10  tagline                     20387 non-null  object        
 11  title                       45345 non-null  object    

In [34]:
#Creamos la columna release_month
df['release_month'] = pd.DatetimeIndex(df['release_date']).month
df['release_month'].head(10)

0    10
1    12
2    12
3    12
4     2
5    12
6    12
7    12
8    12
9    11
Name: release_month, dtype: int64

In [35]:
#convertir la columna release_date a datetime
df['release_date'] = pd.to_datetime(df['release_date'])

In [36]:
#Pasamos el Release_year a str
df['release_year'] = df['release_year'].astype(str)

Realizamos la conversion del dataframe credits

In [37]:
import pandas as pd

# Reemplazamos NaN por una lista compuesta por un diccionario vacío
df['cast'] = df['cast'].apply(lambda x: [] if pd.isna(x) else x)

# Utilizamos una función personalizada que verifique si el valor es una cadena y 
# luego intente convertirla en una lista para finalmente extraer los nombres del cast

def extract_cast_names(value):
    if isinstance(value, str):
        try:
            cast_list = ast.literal_eval(value)
            return [d.get('name') for d in cast_list]
        except (ValueError, SyntaxError):
            return None
    elif isinstance(value, list):
        return [d.get('name') for d in value]
    else:
        return None

#Creamos la columna "cast_names" compuesta por los valores de la clave
#"name" de los diccionarios de las listas de la columna "cast"

df['elenco'] = df['cast'].apply(extract_cast_names)

#Reemplazamos las listas vacías por None
df['elenco'] = df['elenco'].replace([], None)

#Eliminamos la columna "cast" original
df = df.drop('cast', axis=1)

In [38]:
# Reemplazamos NaN por una lista compuesta por un diccionario vacío
df['crew'] = df['crew'].apply(lambda x: [] if pd.isna(x) else x)

# Utilizamos una función personalizada que verifique si el valor es una cadena y 
#luego intente convertirla en una lista para finalmente extraer los nombres del crew

def extract_crew_names(value):
    if isinstance(value, str):
        try:
            crew_list = ast.literal_eval(value)
            return [d.get('name') for d in crew_list]
        except (ValueError, SyntaxError):
            return None
    elif isinstance(value, list):
        return [d.get('name') for d in value]
    else:
        return None
#Creamos la columna "crew_names" compuesta por los valores de la clave
df['directores'] = df['crew'].apply(extract_crew_names)

#Reemplazamos las listas vacías por None
df['directores'] = df['directores'].replace([], None)

#Eliminamos la columna "crew" original
df = df.drop('crew', axis=1)
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,spoken_languages,status,...,collection_poster,collection_backdrop,genre_ids,genre_names,production_companies_ids,production_companies_names,country,release_month,elenco,directores
0,30000000.0,862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,English,Released,...,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[16, 35, 10751]","[Animation, Comedy, Family]",[3],[Pixar Animation Studios],United States of America,10,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J..."
1,65000000.0,8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,English,Released,...,None,None,"[12, 14, 10751]","[Adventure, Fantasy, Family]","[559, 2550, 10201]","[TriStar Pictures, Teitler Film, Interscope Co...",United States of America,12,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Larry J. Franco, Jonathan Hensleigh, James Ho..."
2,0.0,15602.0,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,English,Released,...,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[10749, 35]","[Romance, Comedy]","[6194, 19464]","[Warner Bros., Lancaster Gate]",United States of America,12,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Howard Deutch, Mark Steven Johnson, Mark Stev..."
3,16000000.0,31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,English,Released,...,None,None,"[35, 18, 10749]","[Comedy, Drama, Romance]",[306],[Twentieth Century Fox Film Corporation],United States of America,12,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Forest Whitaker, Ronald Bass, Ronald Bass, Ez..."
4,0.0,11862.0,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,English,Released,...,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,[35],[Comedy],"[5842, 9195]","[Sandollar Productions, Touchstone Pictures]",United States of America,2,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[Alan Silvestri, Elliot Davis, Nancy Meyers, N..."


In [41]:
# Guardamos los cambios en el mismo csv
df.to_csv("movie_dataset_clean.csv", encoding='utf-8-sig', index=False, sep=',') 